<a href="https://colab.research.google.com/github/talpt/pyton/blob/main/MML_Fractal_Tarama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Gerekli kütüphaneleri yükleyin
!pip install git+https://github.com/rongardF/tvdatafeed
!pip install tradingview-screener==2.5.0

import pandas as pd
import numpy as np
from tvDatafeed import TvDatafeed, Interval
from tradingview_screener import get_all_symbols
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

# TvDatafeed giriş
tv = TvDatafeed()

# Türkiye piyasasındaki sembolleri çekme
try:
    Hisseler = get_all_symbols(market='turkey')
    Hisseler = [symbol.replace('BIST:', '') for symbol in Hisseler]
    Hisseler = sorted(Hisseler)
except Exception as e:
    print(f"Sembol listesi alınırken hata oluştu: {e}")
    Hisseler = []

# Murrey Math Hesaplama
def murrey_math(high, low, length, multiplier):
    hi = high.rolling(window=length).max()
    lo = low.rolling(window=length).min()
    chan_range = hi - lo
    multipliers = chan_range * multiplier
    midline = lo + multipliers * 4
    return midline

# Zaman dilimi seçme
def select_interval():
    interval_map = {
        '1': ('15 Dakika', Interval.in_15_minute),
        '2': ('30 Dakika', Interval.in_30_minute),
        '3': ('45 Dakika', Interval.in_45_minute),
        '4': ('1 Saat', Interval.in_1_hour),
        '5': ('2 Saat', Interval.in_2_hour),
        '6': ('4 Saat', Interval.in_4_hour),
        '7': ('Günlük', Interval.in_daily),
        '8': ('Haftalık', Interval.in_weekly),
        '9': ('Aylık', Interval.in_monthly)
    }

    print("Lütfen taramak istediğiniz zaman dilimini seçin:")
    for key, (name, _) in interval_map.items():
        print(f"{key}: {name}")

    while True:
        choice = input("Zaman dilimini seçin (1-9): ").strip()
        if choice in interval_map:
            print(f"Seçilen zaman dilimi: {interval_map[choice][0]}")
            return interval_map[choice][1]
        else:
            print("Geçersiz giriş, lütfen 1 ile 9 arasında bir değer girin.")

# Hisse tarama fonksiyonu
def scan_stocks(interval, length=100, multiplier=0.125):
    results = []
    for symbol in Hisseler:
        try:
            data = tv.get_hist(symbol, exchange='BIST', interval=interval, n_bars=365)
            if data is None or data.empty or 'close' not in data.columns:
                print(f"Veri alınamadı veya eksik sütun: {symbol}")
                continue

            high = data['high']
            low = data['low']
            close = data['close']
            midline = murrey_math(high, low, length, multiplier)

            # Alım sinyali koşulu
            buy_signal = (close > midline) & (close.shift(1) <= midline.shift(1))

            # En son çubukta alım sinyali olup olmadığını kontrol et
            if buy_signal.iloc[-1]:
                results.append({'Symbol': symbol, 'Buy Signal': True})
                print(f"Alım sinyali bulundu: {symbol}")

        except Exception as e:
            print(f"{symbol} için hata: {e}")
    return results

# Kullanıcıdan zaman dilimini seçmesini iste
selected_interval = select_interval()

# Seçilen zaman diliminde tarama yap
print(f"Seçilen zaman diliminde tarama başlıyor...")
results = scan_stocks(selected_interval)

# Sonuçları DataFrame'e dönüştür ve yazdır
if results:
    df = pd.DataFrame(results)
    print("\nAlım Sinyali Olan Hisseler:")
    print(df)
else:
    print("Seçilen zaman diliminde alım sinyali bulunan hisse yok.")


  Cloning https://github.com/rongardF/tvdatafeed to /tmp/pip-req-build-hd1g53tu
  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed /tmp/pip-req-build-hd1g53tu
  Resolved https://github.com/rongardF/tvdatafeed to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py) ... done


Lütfen taramak istediğiniz zaman dilimini seçin:
1: 15 Dakika
2: 30 Dakika
3: 45 Dakika
4: 1 Saat
5: 2 Saat
6: 4 Saat
7: Günlük
8: Haftalık
9: Aylık
Zaman dilimini seçin (1-9): 7
Seçilen zaman dilimi: Günlük
Seçilen zaman diliminde tarama başlıyor...
Alım sinyali bulundu: ATEKS
Alım sinyali bulundu: GMSTR
Alım sinyali bulundu: ISBTR
Alım sinyali bulundu: KLRHO
Alım sinyali bulundu: OZSUB
Alım sinyali bulundu: PENGD
Alım sinyali bulundu: SANEL
Alım sinyali bulundu: SELVA
Alım sinyali bulundu: SMART
Alım sinyali bulundu: TSPOR

Alım Sinyali Olan Hisseler:
  Symbol  Buy Signal
0  ATEKS        True
1  GMSTR        True
2  ISBTR        True
3  KLRHO        True
4  OZSUB        True
5  PENGD        True
6  SANEL        True
7  SELVA        True
8  SMART        True
9  TSPOR        True
